# Instagram: Location Analysis

In [ ]:
# Visual Python: Data Analysis > Import
import numpy as np
import pandas as pd
import time
import re
import os
import random

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
import subprocess
import shutil
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

### Instagram 데이터 수집 함수 생성

In [ ]:
#검색어 페이지 주소 생성 
def insta_search(word):
    url = "https://www.instagram.com/explore/tags/" + str(word)
    return url

In [ ]:
# 첫 번재 게시글 선택
def select_first(driver):
    first = driver.find_elements(By.CSS_SELECTOR, 'div._aagu')[0]
    first.click()
    time.sleep(3)

In [ ]:
# 게시글 정보 가져오기 

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    try:
        content = soup.select('div._a9zs > span')[0].text # html 태그 확인
        #content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
        
    tags = re.findall(r'#[^\s#,\\]+',content)
    date = soup.select('time._aaqe')[0]['datetime'][:10]
    
    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade')[0].findAll('span')[-1].text # html 태그 확인
    except:
        like = 0
    
    try:
        place = soup.select('div._aaql')[0].text
        # html 태그 확인
        #place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data 

In [ ]:
# 다음 게시글 조회
def move_next(driver): 
    right = driver.find_element(By.CSS_SELECTOR, 'div._aaqg._aaqh') # html 태그 확인
    time.sleep(1)
    right.click()
    time.sleep(1.5)

### Chrome Driver 설정

In [ ]:
def chromeWebdriver():
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    options = Options()
    options.add_experimental_option('detach', True)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    
    driver = webdriver.Chrome(service=chrome_service, options=options)
    return driver

In [ ]:
driver = chromeWebdriver()
#driver = webdriver.Chrome(r'C:\Users\user\chromedriver.exe')
driver.get('http://www.instagram.com')
driver.implicitly_wait(3)

In [ ]:
#driver.quit()
#driver.close()

### Instagram 계정 설정

In [ ]:
insta_id = '0ffthe_record'
input_id = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[0] 

input_id.clear()
input_id.send_keys(insta_id)

password = 'han0725!'
input_pw = driver.find_elements(By.CSS_SELECTOR, 'input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

### Instagram 데이터 수집하기

In [ ]:
%%time 

url = insta_search('southkoreatrip')
driver.get(url)
time.sleep(7)

select_first(driver)

results = [ ] 

target = 1000
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(4)
        move_next(driver)
    time.sleep(random.uniform(1, 4))
    if i % 100 == 0:
        print("{0}번 끝".format(i))

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

results_df.to_csv('./data/koreagram_insta_crawling.csv', index=False, encoding='utf-8-sig')

In [ ]:
# Visual Python: Data Analysis > File
result_df = pd.read_csv('./data/southkoreatrip_insta_crawling.csv', encoding='utf-8-sig')
result_df

### N개 이어서 크롤링

In [ ]:
%%time

url = insta_search('koreagram')
driver.get(url)
time.sleep(20)

select_first(driver)

results = [ ] 

# 이전 n(target1)개 그냥 넘기기
target1 = 11700
for i in range(target1):
    move_next(driver)
    if i % 100 == 0:
        print("{0}번 끝".format(i))

# 뒤의 n(target2)개 이어서 크롤링
target2 = 300
for i in range(target2):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)   
    except:
        time.sleep(2)
        move_next(driver)
    time.sleep(random.uniform(1, 4))
    if i % 100 == 0:
        print("{0}번 끝".format(i))
    
print(results[:1])

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']

# 원래 파일 뒤에 이어서 저장
#results_df.to_csv('./southkoreatrip_insta_crawling.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

results_df.to_csv('./koreagram_insta_crawling(1004).csv', index=False, encoding='utf-8-sig')

In [ ]:
result_df = pd.read_csv('./southkoreatrip_insta_crawling.csv', encoding='utf-8-sig')
result_df

### 위치정보 가져오기

In [ ]:
location_counts = results_df['place'].value_counts()
print(location_counts)

In [ ]:
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.to_csv('./koreagram_location_counts.csv', encoding='utf-8-sig')

In [ ]:
locations = list(location_counts.index)
print(locations)

In [ ]:
import requests 

searching = '창경궁'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)

headers = {"Authorization": "KakaoAK 7b79badcab2f1b9bd186f82a4ade78e2"}

places = requests.get(url, headers = headers).json()['documents']
print(places)

In [ ]:
def find_places(searching):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {"Authorization": "KakaoAK 7b79badcab2f1b9bd186f82a4ade78e2"}
    
    places = requests.get(url, headers = headers).json()['documents']
    place = places[0]
    name = place['place_name']

    x = place['x']
    y = place['y']
    data = name, x, y, searching

    return data

In [ ]:
print(find_places('창경궁'))

In [ ]:
import time
from tqdm import tqdm

locations_inform = []
for location in tqdm(locations):
    try:
        data = find_places(location)
        locations_inform.append(data)
        time.sleep(0.5)
    except:
        pass

locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['네이버위치명', '경도', '위도', '인스타위치명']
locations_inform_df.to_csv('./data/koreagram_location_final.csv', index=False, encoding='utf-8-sig')

### 위치 데이터 시각화 (Plotly)

In [ ]:
# Visual Python: Logic > code
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
locations_inform_df

In [ ]:
locations_inform_df.dtypes

In [ ]:
locations_inform_df = locations_inform_df[['경도','위도']].astype('float64')

In [ ]:
fig = px.scatter_mapbox(locations_inform_df, lat='위도', lon='경도', size_max=400, zoom=6)
fig.update_layout(mapbox_style = 'open-street-map')
fig.show()

In [ ]:
#End of File